In [356]:
from core.semcor_bert_pipeline import *
import pandas as pd
from core.metrics import cosine_sim, centroid

In [357]:
df = pd.read_csv('../data/rabagliati_2013_stimuli.csv', sep = '\t')
df = df[~df['target'].isin(['moose', 'mousse'])] #BERT tokenization issues
model = initialize_masking_lm()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [358]:
df['pos'] = ['n'] * len(df.index)
df['word'] = df['target']
word_pos_pairs = df[['word', 'pos']].drop_duplicates()
word_pos_pairs.to_csv('../data/r_13_for_semcor.csv')

In [359]:
mask_results = []
for i in range(len(df.index)):
    row = df.iloc[i]
    s1, s2, target_token = row['s1'], row['s2'], row['target']
    indexed_tokens, tokenized_text = preprocess(s1, target_token, s2 = s2, masking = True)
    embed, preds = mask_predictions(indexed_tokens, model, tokenizer)
    mask_results.append({'mask_embed': embed, 'predicted_tokens': preds[1], 'predicted_probs': preds[0]})

In [360]:
both_sense_types = []
one_sense_types = []
available_sense = []
missing_words = ['lamb', 'herbs', 'turkey', 'tuna', 'shrimp', 'cards']
for w in word_pos_pairs.iterrows():
    w = w[1]
    word, pos = w['word'], w['pos']
    if word == 'cards':
        semcor_word = 'card'
    elif word == 'herbs':
        semcor_word = 'herb'
    else:
        semcor_word = word
    try:
        type_data = load_data(semcor_word, pos, 'semcor')
        word_senses = df[df['target'] == word]['wn_sense'].unique()
        contains_both_senses = lambda l1, l2: all([s in l2 for s in l1])
        if contains_both_senses(word_senses, type_data['sense_labels']):
            both_sense_types.append(word)
        common_senses = set(word_senses).intersection(set(type_data['sense_labels']))
        if len(common_senses) == 1 and word not in ['son', 'sun', 'night', 'knight']:
            one_sense_types.append(word)
            available_sense.append(common_senses.pop())
            
    except:
        pass

In [361]:
len(both_sense_types)

15

In [362]:
len(one_sense_types)

8

This looks like a little less than half the types. Let's work on these first, then get to the edge cases (1 sense, glass.n, homophones)

In [363]:
both_sense_types

['chicken',
 'fish',
 'herbs',
 'letter',
 'button',
 'bow',
 'line',
 'bat',
 'nail',
 'pitcher',
 'band',
 'sun',
 'son',
 'knight',
 'night']

In [364]:
one_sense_types

['lamb', 'turkey', 'duck', 'glasses', 'mouse', 'roll', 'cards', 'bark']

In [365]:
def nearest_neighbor(query, s1, s2, sense_names):
    s1_sims = [cosine_sim(query, e) for e in s1]
    s2_sims = [cosine_sim(query, e) for e in s2]
    if max(s1_sims) > max(s2_sims):
        return sense_names[0]
    if max(s1_sims) < max(s2_sims):
        return sense_names[1]
def centroid_pred(query, c1, c2, sense_names):
    c1_sim = cosine_sim(query, c1)
    c2_sim = cosine_sim(query, c2)
    if c1_sim > c2_sim:
        return sense_names[0]
    if c1_sim < c2_sim:
        return sense_names[1]

In [366]:
def one_sense_preds(query_embeds, sense_embeds, data_sense, other_sense, method):
    #method is either "nn" or "centroid"
    if method == "nn":
        cos_sims = [max([cosine_sim(q, s) for s in sense_embeds]) for q in query_embeds]
    if method == 'centroid':
        c = centroid(sense_embeds)
        cos_sims = [cosine_sim(q, c) for q in query_embeds]
    sense_preds = ["" for i in range(4)]
    sorted_indices = np.argsort(cos_sims)
    least_sim, most_sim = sorted_indices[:2], sorted_indices[2:]
    for i in range(len(sense_preds)):
        if i in least_sim:
            sense_preds[i] = other_sense
        if i in most_sim:
            sense_preds[i] = data_sense
    return sense_preds


In [368]:
reg_cases = df[df['target'].isin(both_sense_types[:-4])]
e_preds = []
c_preds = []
for w in reg_cases['target'].unique():
    indices = df[df['target'] == w].index
    senses = df[df['target'] == w]['wn_sense'].unique()
    mask_embeddings = [mask_results[i]['mask_embed'] for i in indices]
    if w == 'herbs':
        semcor_word = 'herb'
    else:
        semcor_word = w
    token_data = load_data(semcor_word, 'n', 'semcor')
    semcor_embeddings = np.array(token_data['embeddings'])
    num_embeddings = len(semcor_embeddings)
    s1_embeds = semcor_embeddings[[i for i in range(num_embeddings) if token_data['sense_labels'][i] == senses[0]]]
    s2_embeds = semcor_embeddings[[i for i in range(num_embeddings) if token_data['sense_labels'][i] == senses[1]]]
    s1_centroid, s2_centroid = centroid(s1_embeds), centroid(s2_embeds)
    e_preds += [nearest_neighbor(e, s1_embeds, s2_embeds, senses) for e in mask_embeddings]
    c_preds += [centroid_pred(e, s1_centroid, s2_centroid, senses) for e in mask_embeddings]
    

In [369]:
homophones = [('son', 'sun'), ('night', 'knight')]
for p in homophones:
    t1, t2 = load_data(p[0], 'n', 'semcor'), load_data(p[1], 'n', 'semcor')
    s1, s2 = df[df['target'] == p[0]]['wn_sense'].values[0], df[df['target'] == p[1]]['wn_sense'].values[0]
    mask_indices = list(df[df['target'] == p[0]].index) + list(df[df['target'] == p[1]].index)
    mask_embeddings = [mask_results[i]['mask_embed'] for i in mask_indices]
    s1_embeds = np.array(t1['embeddings'])[[i for i in range(len(t1['embeddings'])) if t1['sense_labels'][i] \
                                            == s1]]
    s2_embeds = np.array(t2['embeddings'])[[i for i in range(len(t2['embeddings'])) if t2['sense_labels'][i] \
                                            == s2]]
    e_preds += [nearest_neighbor(e, s1_embeds, s2_embeds, [s1, s2]) for e in mask_embeddings]
    s1_centroid, s2_centroid = centroid(s1_embeds), centroid(s2_embeds)
    c_preds += [centroid_pred(e, s1_centroid, s2_centroid, [s1, s2]) for e in mask_embeddings]
    



In [370]:
two_senses = df[df['target'].isin(both_sense_types)]
two_senses['nn_preds'] = e_preds
two_senses['centroid_preds'] = c_preds

/anaconda3/envs/bert/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/envs/bert/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [372]:
one_sense = df[df['target'].isin(one_sense_types)]
one_sense_nn = []
one_sense_centroid = []

for w, s in zip(one_sense_types, available_sense):
    indices = df[df['target'] == w].index
    senses = df[df['target'] == w]['wn_sense'].unique()
    mask_embeddings = [mask_results[i]['mask_embed'] for i in indices]
    if w == 'cards':
        semcor_word = 'card'
    else:
        semcor_word = word
    token_data = load_data(w, 'n', 'semcor')
    semcor_embeddings = np.array(token_data['embeddings'])
    num_embeddings = len(semcor_embeddings)
    sense_embeds = semcor_embeddings[[i for i in range(num_embeddings) if token_data['sense_labels'][i] == s]]
    other = [sen for sen in senses if sen != s][0]
    
    if w == 'glasses':
        glass_embeds = load_data('glass', 'n', 'semcor')
        item_embeds = np.array(glass_embeds['embeddings'])[[i for i in range(len(glass_embeds['embeddings'])) \
                                              if glass_embeds['sense_labels'][i] == other]]
        one_sense_nn += [nearest_neighbor(e, sense_embeds, item_embeds, [s1, s2]) for e in mask_embeddings]
        s1_centroid, s2_centroid = centroid(sense_embeds), centroid(item_embeds)
        one_sense_centroid += [centroid_pred(e, s1_centroid, s2_centroid, [s, other]) for e in mask_embeddings]

    else:
        one_sense_nn += one_sense_preds(mask_embeddings, semcor_embeddings, s, other, 'nn')
        one_sense_centroid += one_sense_preds(mask_embeddings, semcor_embeddings, s, other, 'nn')
    
one_sense['nn_preds'] = one_sense_nn
one_sense['centroid_preds'] = one_sense_centroid

/anaconda3/envs/bert/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/anaconda3/envs/bert/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [373]:
all_data = pd.concat([two_senses, one_sense])
def accuracy_by_rel_type(df, rel_type, pred_type):
    df = df[df['rel_type'] == rel_type]
    return sum(df[pred_type] == df['wn_sense']) / len(df.index)

In [383]:
sum(all_data['centroid_preds'] == all_data['wn_sense']) / len(all_data.index)

0.6190476190476191

In [384]:
accuracy_by_rel_type(all_data, 'reg_pol', 'centroid_preds')

0.4583333333333333

In [385]:
accuracy_by_rel_type(all_data, 'irreg_pol', 'centroid_preds')

0.8125

In [386]:
accuracy_by_rel_type(all_data, 'hom', 'centroid_preds')

0.5357142857142857

In [388]:
pd.DataFrame({"rel_type": df['rel_type'].unique(), "centroid_acc": [accuracy_by_rel_type(all_data,
                                                                                        rt, 'centroid_preds') for rt in df['rel_type'].unique()],
             "nn_acc": [accuracy_by_rel_type(all_data, rt, 'nn_preds') for rt in df['rel_type'].unique()]})

,rel_type,centroid_acc,nn_acc
0,reg_pol,0.458333,0.500000
1,irreg_pol,0.812500,0.656250
2,hom,0.535714,0.642857
